# Using TensorBoard to track metrics, images, audio and text

In this notebook we show how to use the TensorBoard extension which allows to keep track of training runs and data associated with each epoch (e.g. loss, accuracies, predicted/generated outputs, activation statistics etc.)

First we will look at tensorflow's summary writer and what it can (and should) be used for. In a separate notebook, we will look at an example of how this can be used inside a subclassed model to keep track of the training and validation loss, as well as accuracies, also using keras metrics.

The summary writer can be used to log scalars, feature maps (e.g. batches of generated images or activations), histograms, audio (batches of 1D sequences), and text.

In [7]:
import tensorflow as tf
import math
import datetime

We start by loading the tensorboard extension. This only applies if you're using a notebook and want to show the tensorboard inside the notebook.

In [5]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


For the sake of demonstration, we iterate over 100 steps and try to store a loss, some randomly generated images, an audio tensor, text, and a histogram for each step. We do not yet use a deep learning model, we only show the code necessary to store different kinds of data to log files that can be read by the TensorBoard.

In [12]:
# define file-path for log file
file_path = "test_logs/test"

# define the tf file-writer (we usually use a separate one for train and validation)
summary_writer = tf.summary.create_file_writer(file_path)

loss_function = tf.keras.losses.MeanSquaredError()

# write 100 logs for loss

for i in range(100):

    # CREATE DATA OF DIFFERENT KINDS
    
    # compute loss (here targets and predictions would come from the data and the model)
    targets = tf.constant([0.3,0.3,-0.8])
    predictions = targets + tf.random.normal(shape=targets.shape, stddev=100/(i+1)) # decreasing noise
    loss = loss_function(targets,predictions)
    
    # image batch (these would be obtained from the model)
    image_batch = tf.random.uniform(shape=(32,28,28,1),dtype=tf.float32)
    
    # audio batch (would be obtained from the model but here it's just a hard coded sine wave of 110hz)
    x = 2* math.pi*tf.cast(tf.linspace(0,32000*5, 32000*5), tf.float32)*110/32000
    x = tf.expand_dims(x, axis=0) # add batch dimension
    x = tf.expand_dims(x, axis=-1) # add last dimension
    x = tf.repeat(x, 32, axis=0) # repeat to have a batch of 32
    audio_batch = tf.math.sin(x) # obtain sine wave
    
    # text (this would be the output of a language model after one training epoch)
    text = tf.constant("This is the sampled output of a language model")
    
    # histogram (e.g. of activations of a dense layer during training)
    activations_batch = tf.random.normal(shape=(32,20,1))
    #in_activations = tf.reduce_min(activations_batch, axis=None)
    #max_activations = tf.reduce_max(activations_batch, axis=None)
    #histogram = tf.histogram_fixed_width_bins(activations_batch, 
    #                                          value_range=[min_activations, max_activations])
    histogram = activations_batch #tf.reduce_mean(activations_batch,axis=
    # WRITE THE DATA TO A LOG FILE USING A SUMMARY WRITER
    with summary_writer.as_default():
        
        # save the loss scalar for the "epoch"
        tf.summary.scalar(name="loss", data=loss, step=i)
        
        # save a batch of images for this epoch (have to be between 0 and 1)
        tf.summary.image(name="generated_images",data = image_batch, step=i, max_outputs=32)
        
        # save the batch of audio for this epoch
        tf.summary.audio(name="generated_audio", data = audio_batch, 
                         sample_rate = 32000, step=i, max_outputs=32)
        
        # save the generated text for that epoch
        tf.summary.text(name="generated_text", data = text, step=i)
        
        # save a histogram (e.g. of activations in a layer)
        tf.summary.histogram(name="layer_N_activations", data = activations_batch, step=i)

# Inspect the logged data in the TensorBoard

We can look at the images, audio, text, histograms and plots for each time-step. 

For plots under the "scalars" section, we can control the amount of smoothing for the plots. This allows us to visually judge whether the loss is decreasing even in the presence of strong oscillations.

# Open the tensorboard

- If you're not using a notebook, activate your environment in your terminal and type tensorboard --logdir test_logs/ where test_logs is the folder that contains the logs.

In [11]:
%tensorboard --logdir test_logs/

Reusing TensorBoard on port 6007 (pid 21473), started 2:08:56 ago. (Use '!kill 21473' to kill it.)

# Conclusion

- We can keep track of many things and analyze them already while a model is still training.

- We can save activations and also gradients during training as images or histograms, which can be super helpful for debugging!

- We can monitor model performance either quantitatively using scalar measures or qualitatively by logging predictions (image outputs, audio outputs or text outputs)


Next: **Full training loop**, using Tensorboard with the model class incl. keras metrics